In [18]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import geopandas as gpd
from shapely.geometry import Point

In [30]:
col_name = ['FL_DATE','AIRLINE','AIRLINE_DOT','AIRLINE_CODE','DOT_CODE','FL_NUMBER','ORIGIN','ORIGIN_CITY','DEST','DEST_CITY','CRS_DEP_TIME','DEP_TIME','DEP_DELAY','TAXI_OUT','WHEELS_OFF','WHEELS_ON','TAXI_IN','CRS_ARR_TIME','ARR_TIME','ARR_DELAY','CANCELLED','CANCELLATION_CODE','DIVERTED','CRS_ELAPSED_TIME','ELAPSED_TIME','AIR_TIME','DISTANCE','DELAY_DUE_CARRIER','DELAY_DUE_WEATHER','DELAY_DUE_NAS','DELAY_DUE_SECURITY','DELAY_DUE_LATE_AIRCRAFT']
flights = pd.read_csv('./data/flights_sample_3m.csv', names=col_name)
flights.head(10)

,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2120,2146.0,26.0,18.0,2204.0,2326.0,3.0,2310,2329.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-05,Spirit Air Lines,Spirit Air Lines: NK,NK,20416.0,618.0,MSY,"New Orleans, LA",MCO,"Orlando, FL",...,0.0,96.0,89.0,70.0,551.0,NaN,NaN,NaN,NaN,NaN
2,2023-06-17,Endeavor Air Inc.,Endeavor Air Inc.: 9E,9E,20363.0,5158.0,JFK,"New York, NY",BTV,"Burlington, VT",...,0.0,82.0,79.0,45.0,266.0,NaN,NaN,NaN,NaN,NaN
3,2022-05-30,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790.0,2251.0,DTW,"Detroit, MI",BDL,"Hartford, CT",...,0.0,97.0,88.0,72.0,549.0,NaN,NaN,NaN,NaN,NaN
4,2023-04-28,American Airlines Inc.,American Airlines Inc.: AA,AA,19805.0,1708.0,EWR,"Newark, NJ",MIA,"Miami, FL",...,0.0,187.0,238.0,195.0,1085.0,22.0,0.0,51.0,0.0,0.0
5,2020-10-24,SkyWest Airlines Inc.,SkyWest Airlines Inc.: OO,OO,20304.0,4276.0,BJI,"Bemidji, MN",MSP,"Minneapolis, MN",...,0.0,60.0,56.0,38.0,199.0,NaN,NaN,NaN,NaN,NaN
6,2019-09-12,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790.0,931.0,LGA,"New York, NY",IAH,"Houston, TX",...,0.0,232.0,246.0,174.0,1416.0,59.0,0.0,14.0,0.0,0.0
7,2019-01-04,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977.0,1183.0,EWR,"Newark, NJ",RSW,"Fort Myers, FL",...,0.0,195.0,184.0,158.0,1068.0,NaN,NaN,NaN,NaN,NaN
8,2023-07-15,Southwest Airlines Co.,Southwest Airlines Co.: WN,WN,19393.0,424.0,PIT,"Pittsburgh, PA",LAS,"Las Vegas, NV",...,0.0,265.0,271.0,258.0,1910.0,7.0,0.0,6.0,0.0,16.0
9,2022-10-26,PSA Airlines Inc.,PSA Airlines Inc.: OH,OH,20397.0,5663.0,DCA,"Washington, DC",MSY,"New Orleans, LA",...,0.0,182.0,155.0,137.0,969.0,NaN,NaN,NaN,NaN,NaN


In [31]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354626 entries, 0 to 2354625
Data columns (total 32 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 float64
 5   FL_NUMBER                float64
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             float64
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             float64
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  CANCELLA

In [33]:
miss_ser = flights.isna().sum()

miss_val = pd.DataFrame({
    'Missing Values':miss_ser.values,
    'Miss rate':round((miss_ser/flights.shape[0])*100,4).astype(str) + '%',
    'Datatype':flights.dtypes
})

miss_val.sort_values(by='Miss rate', ascending=False)

,Missing Values,Miss rate,Datatype
CANCELLATION_CODE,2292476,97.3605%,object
DELAY_DUE_LATE_AIRCRAFT,1935937,82.2184%,float64
DELAY_DUE_SECURITY,1935937,82.2184%,float64
DELAY_DUE_NAS,1935937,82.2184%,float64
DELAY_DUE_WEATHER,1935937,82.2184%,float64
DELAY_DUE_CARRIER,1935937,82.2184%,float64
AIR_TIME,67724,2.8762%,float64
ELAPSED_TIME,67724,2.8762%,float64
ARR_DELAY,67723,2.8762%,float64
WHEELS_ON,62791,2.6667%,float64


The data presented in the table clearly indicates that over 97% of the feature labeled CANCELLATION_CODE is absent, and this feature is stored as a string object. Since it's not feasible to deduce this information without explicit details, we will remove this feature to prevent any potential biases in assumptions.

Additionally, the other tables related to delays show that more than 80% of the entries are missing. Given that these values are numerical, we can reasonably infer that a missing entry indicates no delays occurred. Therefore, we will replace these missing values with 0 to simplify calculations while analyzing the delay data.

Lastly, the missing values in the other flight logistics characteristics account for only about 3%. These values are significant enough to warrant consideration, so we will employ kNN imputation to fill in the missing data, utilizing the mean from the 'k' nearest neighbors for this purpose.

In [34]:
flights.drop('CANCELLATION_CODE', inplace=True, axis=1)
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354626 entries, 0 to 2354625
Data columns (total 31 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   FL_DATE                  object 
 1   AIRLINE                  object 
 2   AIRLINE_DOT              object 
 3   AIRLINE_CODE             object 
 4   DOT_CODE                 float64
 5   FL_NUMBER                float64
 6   ORIGIN                   object 
 7   ORIGIN_CITY              object 
 8   DEST                     object 
 9   DEST_CITY                object 
 10  CRS_DEP_TIME             float64
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  CRS_ARR_TIME             float64
 18  ARR_TIME                 float64
 19  ARR_DELAY                float64
 20  CANCELLED                float64
 21  DIVERTED

In [35]:
delay_cols = ['DELAY_DUE_LATE_AIRCRAFT', 'DELAY_DUE_SECURITY', 'DELAY_DUE_NAS',
              'DELAY_DUE_WEATHER', 'DELAY_DUE_CARRIER']

flights[delay_cols] = flights[delay_cols].fillna(0)
flights.isna().sum()

,0
FL_DATE,0
AIRLINE,0
AIRLINE_DOT,0
AIRLINE_CODE,0
DOT_CODE,0
FL_NUMBER,0
ORIGIN,0
ORIGIN_CITY,0
DEST,0
DEST_CITY,0


Given the substantial size of the dataset, performing any form of imputation or data cleansing could result in a significant computational burden.

Therefore, we will extract a subset of the dataset that corresponds to the airline with the most operating flights. This process involves counting the instances for each airline represented in the dataset and selecting the one that has the greatest number of entries.

In [36]:
flights.loc[:,['AIRLINE_CODE','AIRLINE','AIRLINE_DOT']].value_counts()

,,,count
AIRLINE_CODE,AIRLINE,AIRLINE_DOT,
WN,Southwest Airlines Co.,Southwest Airlines Co.: WN,452817
DL,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,309754
AA,American Airlines Inc.,American Airlines Inc.: AA,301111
OO,SkyWest Airlines Inc.,SkyWest Airlines Inc.: OO,270003
UA,United Air Lines Inc.,United Air Lines Inc.: UA,199464
YX,Republic Airline,Republic Airline: YX,112400
MQ,Envoy Air,Envoy Air: MQ,95019
B6,JetBlue Airways,JetBlue Airways: B6,88689
9E,Endeavor Air Inc.,Endeavor Air Inc.: 9E,88151


Based on the observations mentioned earlier, it is clear that __Southwest Airlines__ accounts for the largest portion of our dataset, representing approximately `20%` of the total entries. This abundance of data allows us to concentrate our analysis on a single airline, facilitating more accurate
insights specific to this airline operator.

In [37]:
fl_sw = flights.loc[(flights.loc[:,'AIRLINE_CODE'] == 'WN'), :].reset_index(drop=True)
display(fl_sw.head().T)
display(fl_sw.loc[:,'AIRLINE'].value_counts())
display(fl_sw.loc[:,'AIRLINE_DOT'].value_counts())
display(fl_sw.loc[:,'AIRLINE_CODE'].value_counts())

,0,1,2,3,4
FL_DATE,2023-07-15,2022-04-30,2020-12-08,2023-05-13,2023-03-11
AIRLINE,Southwest Airlines Co.,Southwest Airlines Co.,Southwest Airlines Co.,Southwest Airlines Co.,Southwest Airlines Co.
AIRLINE_DOT,Southwest Airlines Co.: WN,Southwest Airlines Co.: WN,Southwest Airlines Co.: WN,Southwest Airlines Co.: WN,Southwest Airlines Co.: WN
AIRLINE_CODE,WN,WN,WN,WN,WN
DOT_CODE,19393.0,19393.0,19393.0,19393.0,19393.0
FL_NUMBER,424.0,3818.0,2583.0,1404.0,889.0
ORIGIN,PIT,HOU,MCO,ELP,BNA
ORIGIN_CITY,"Pittsburgh, PA","Houston, TX","Orlando, FL","El Paso, TX","Nashville, TN"
DEST,LAS,MAF,SJU,AUS,FLL
DEST_CITY,"Las Vegas, NV","Midland/Odessa, TX","San Juan, PR","Austin, TX","Fort Lauderdale, FL"


,count
AIRLINE,
Southwest Airlines Co.,452817


,count
AIRLINE_DOT,
Southwest Airlines Co.: WN,452817


,count
AIRLINE_CODE,
WN,452817


Having extracted and created a subset of the dataset for Southwest Airlines, we can now perform our exploratory data analysis on the refined dataset.

In [38]:
fl_sw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452817 entries, 0 to 452816
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   FL_DATE                  452817 non-null  object 
 1   AIRLINE                  452817 non-null  object 
 2   AIRLINE_DOT              452817 non-null  object 
 3   AIRLINE_CODE             452817 non-null  object 
 4   DOT_CODE                 452817 non-null  float64
 5   FL_NUMBER                452817 non-null  float64
 6   ORIGIN                   452817 non-null  object 
 7   ORIGIN_CITY              452817 non-null  object 
 8   DEST                     452817 non-null  object 
 9   DEST_CITY                452817 non-null  object 
 10  CRS_DEP_TIME             452817 non-null  float64
 11  DEP_TIME                 437591 non-null  float64
 12  DEP_DELAY                437591 non-null  float64
 13  TAXI_OUT                 437532 non-null  float64
 14  WHEE

Let's focus on addressing the columns with missing values by applying the __median imputation method__.

In [39]:
miss_cols = [col for col in fl_sw.columns if fl_sw[col].isna().any()]
miss_cols

['DEP_TIME',
 'DEP_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'ARR_TIME',
 'ARR_DELAY',
 'ELAPSED_TIME',
 'AIR_TIME']

In [40]:
imp = SimpleImputer(strategy='median', copy=True)
fl_sw[miss_cols] = imp.fit_transform(fl_sw[miss_cols])
fl_sw.isna().sum()

,0
FL_DATE,0
AIRLINE,0
AIRLINE_DOT,0
AIRLINE_CODE,0
DOT_CODE,0
FL_NUMBER,0
ORIGIN,0
ORIGIN_CITY,0
DEST,0
DEST_CITY,0


In order to prepare the dataset for analysis, we must adjust the data types of certain features to align with their provided descriptions. Additionally, we will remove some string columns that are redundant since their information is already represented as codes for each entry. Lastly, we will organize the variables in chronological sequence and check for any missing readings.

In [41]:
fl_sw.drop(['AIRLINE','AIRLINE_DOT','ORIGIN_CITY','DEST_CITY'], inplace=True, axis=1)
fl_sw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452817 entries, 0 to 452816
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   FL_DATE                  452817 non-null  object 
 1   AIRLINE_CODE             452817 non-null  object 
 2   DOT_CODE                 452817 non-null  float64
 3   FL_NUMBER                452817 non-null  float64
 4   ORIGIN                   452817 non-null  object 
 5   DEST                     452817 non-null  object 
 6   CRS_DEP_TIME             452817 non-null  float64
 7   DEP_TIME                 452817 non-null  float64
 8   DEP_DELAY                452817 non-null  float64
 9   TAXI_OUT                 452817 non-null  float64
 10  WHEELS_OFF               452817 non-null  float64
 11  WHEELS_ON                452817 non-null  float64
 12  TAXI_IN                  452817 non-null  float64
 13  CRS_ARR_TIME             452817 non-null  float64
 14  ARR_

It is evident that the _flight date_ feature is currently categorized as an `'object'` datatype, while it should actually be in the `'datetime'` format. The remaining datatypes appear to align with their respective feature descriptions.

In [42]:
fl_sw['FL_DATE'] = pd.to_datetime(fl_sw['FL_DATE'], format='%Y-%m-%d')
fl_sw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452817 entries, 0 to 452816
Data columns (total 27 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   FL_DATE                  452817 non-null  datetime64[ns]
 1   AIRLINE_CODE             452817 non-null  object        
 2   DOT_CODE                 452817 non-null  float64       
 3   FL_NUMBER                452817 non-null  float64       
 4   ORIGIN                   452817 non-null  object        
 5   DEST                     452817 non-null  object        
 6   CRS_DEP_TIME             452817 non-null  float64       
 7   DEP_TIME                 452817 non-null  float64       
 8   DEP_DELAY                452817 non-null  float64       
 9   TAXI_OUT                 452817 non-null  float64       
 10  WHEELS_OFF               452817 non-null  float64       
 11  WHEELS_ON                452817 non-null  float64       
 12  TAXI_IN         

We have noticed that the entries in the date column are inconsistent and require chronological organization. This can be achieved with the `sort_values` method, which will sort the data automatically if the feature is formatted as date-time.

In [43]:
fl_sw = fl_sw.sort_values(by='FL_DATE').reset_index(drop=True)
fl_sw

,FL_DATE,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-01,WN,19393.0,1308.0,ATL,BNA,1615.0,1648.0,33.0,14.0,...,0.0,70.0,57.0,38.0,214.0,5.0,0.0,0.0,0.0,15.0
1,2019-01-01,WN,19393.0,1719.0,PBI,BWI,700.0,655.0,-5.0,7.0,...,0.0,145.0,120.0,110.0,883.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-01,WN,19393.0,165.0,TPA,MDW,2010.0,2000.0,-10.0,8.0,...,0.0,160.0,146.0,132.0,997.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-01,WN,19393.0,738.0,MSY,LAS,2050.0,2103.0,13.0,8.0,...,0.0,235.0,207.0,197.0,1500.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-01,WN,19393.0,4.0,HOU,DAL,700.0,700.0,0.0,6.0,...,0.0,60.0,51.0,38.0,239.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452812,2023-08-31,WN,19393.0,4511.0,MEM,DAL,1550.0,1555.0,5.0,13.0,...,0.0,80.0,77.0,59.0,423.0,0.0,0.0,0.0,0.0,0.0
452813,2023-08-31,WN,19393.0,2847.0,TPA,HOU,1245.0,1308.0,23.0,18.0,...,0.0,140.0,131.0,104.0,781.0,0.0,0.0,0.0,0.0,0.0
452814,2023-08-31,WN,19393.0,296.0,BWI,ISP,2210.0,2225.0,15.0,10.0,...,0.0,65.0,63.0,48.0,220.0,0.0,0.0,0.0,0.0,0.0
452815,2023-08-31,WN,19393.0,2924.0,DTW,BWI,615.0,608.0,-7.0,10.0,...,0.0,85.0,74.0,59.0,409.0,0.0,0.0,0.0,0.0,0.0


Based on the review conducted, it appears that the data is complete and ready for our analysis. To further ensure data integrity, we can also verify if there are any duplicate entries present in the dataset.

In [44]:
fl_sw.duplicated().value_counts()

,count
False,452817


Having obtained the final cleaned dataset, we can now perform a distribution analysis to better comprehend the data characteristics, as well as carry out various statistical tests.

Specifically, we will examine daily logistical operations through metrics such as taxiing durations, delays in arrivals and departures, and overall flight times for all flights operated by Southwest Airlines.

In [45]:
fl_sw.describe()

,FL_DATE,DOT_CODE,FL_NUMBER,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
count,452817,452817.0,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,...,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000,452817.000000
mean,2021-05-21 01:57:23.396339456,19393.0,2076.892652,1329.565531,1339.954240,10.431097,12.163243,1358.834891,1457.822330,5.481912,...,0.001952,128.077479,120.102116,102.361643,740.469508,3.186031,0.237745,1.451008,0.032324,4.574819
min,2019-01-01 00:00:00,19393.0,1.000000,500.000000,1.000000,-52.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,35.000000,22.000000,13.000000,73.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2020-01-14 00:00:00,19393.0,921.000000,910.000000,924.000000,-3.000000,9.000000,936.000000,1050.000000,3.000000,...,0.000000,85.000000,78.000000,61.000000,386.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2021-07-10 00:00:00,19393.0,1818.000000,1325.000000,1331.000000,0.000000,11.000000,1342.000000,1502.000000,4.000000,...,0.000000,115.000000,108.000000,90.000000,639.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2022-08-28 00:00:00,19393.0,2843.000000,1735.000000,1741.000000,12.000000,14.000000,1753.000000,1902.000000,6.000000,...,0.000000,155.000000,147.000000,129.000000,972.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2023-08-31 00:00:00,19393.0,6999.000000,2355.000000,2400.000000,602.000000,177.000000,2400.000000,2400.000000,171.000000,...,1.000000,480.000000,488.000000,409.000000,2979.000000,576.000000,581.000000,457.000000,366.000000,597.000000
std,NaN,0.0,1495.851370,484.363447,490.494227,29.659842,6.101028,490.522376,532.340965,4.697197,...,0.044141,56.450910,55.595765,54.435044,459.897068,15.217523,5.303341,8.979150,1.665930,18.868035


In [46]:
fl_sw_op = fl_sw.groupby(by=['FL_DATE'])[['TAXI_IN','TAXI_OUT', 'CRS_ELAPSED_TIME', 'AIR_TIME', 'ELAPSED_TIME','ARR_DELAY','DEP_DELAY', 'DELAY_DUE_CARRIER',
                                          'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT']].mean()
fl_sw_op

,TAXI_IN,TAXI_OUT,CRS_ELAPSED_TIME,AIR_TIME,ELAPSED_TIME,ARR_DELAY,DEP_DELAY,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
FL_DATE,,,,,,,,,,,,
2019-01-01,5.636667,12.116667,130.133333,105.886667,123.670000,5.946667,12.473333,2.636667,0.223333,1.536667,0.063333,5.553333
2019-01-02,5.823718,12.384615,131.041667,107.163462,125.371795,12.160256,17.830128,4.987179,0.076923,1.868590,0.028846,7.490385
2019-01-03,5.265781,12.033223,125.963455,100.514950,117.843854,5.850498,14.116279,4.016611,0.000000,0.697674,0.000000,4.963455
2019-01-04,5.414474,12.210526,131.250000,105.217105,122.842105,-2.345395,6.062500,2.644737,0.000000,0.611842,0.000000,1.835526
2019-01-05,5.018605,12.302326,134.581395,108.139535,125.511628,-1.855814,7.088372,1.465116,0.116279,1.455814,0.000000,2.772093
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27,5.692308,13.760684,119.829060,95.974359,115.415954,5.957265,10.185185,2.569801,0.236467,2.316239,0.000000,5.225071
2023-08-28,5.570571,13.213213,124.309309,100.450450,119.243243,1.468468,6.663664,2.003003,0.420420,1.405405,0.000000,3.369369
2023-08-29,5.153846,12.766382,122.564103,97.475783,115.507123,-2.316239,3.726496,1.193732,0.210826,1.031339,0.000000,2.276353


In [49]:
us_airports = pd.read_csv('./data/us_airports.csv')
us_airports.head(10)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,country_name,iso_country,...,scheduled_service,gps_code,icao_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,3632,KLAX,large_airport,Los Angeles International Airport,33.942501,-118.407997,125.0,NaN,United States,US,...,1,KLAX,KLAX,LAX,LAX,https://www.flylax.com/,https://en.wikipedia.org/wiki/Los_Angeles_Inte...,Tom Bradley,1335475,2024-04-02T16:36:13+00:00
1,3754,KORD,large_airport,Chicago O'Hare International Airport,41.978600,-87.904800,680.0,NaN,United States,US,...,1,KORD,KORD,ORD,ORD,https://www.flychicago.com/ohare/home/pages/de...,https://en.wikipedia.org/wiki/O'Hare_Internati...,"CHI, Orchard Place",1503175,2024-03-09T23:28:49+00:00
2,3622,KJFK,large_airport,John F Kennedy International Airport,40.639447,-73.779317,13.0,NaN,United States,US,...,1,KJFK,KJFK,JFK,JFK,https://www.jfkairport.com/,https://en.wikipedia.org/wiki/John_F._Kennedy_...,"Manhattan, New York City, NYC, Idlewild, IDL, ...",1052075,2022-10-18T18:49:55+00:00
3,3384,KATL,large_airport,Hartsfield Jackson Atlanta International Airport,33.636700,-84.428101,1026.0,NaN,United States,US,...,1,KATL,KATL,ATL,ATL,http://www.atlanta-airport.com/,https://en.wikipedia.org/wiki/Hartsfield–Jacks...,NaN,2002475,2024-04-02T16:26:01+00:00
4,3878,KSFO,large_airport,San Francisco International Airport,37.619806,-122.374821,13.0,NaN,United States,US,...,1,KSFO,KSFO,SFO,SFO,http://www.flysfo.com/,https://en.wikipedia.org/wiki/San_Francisco_In...,"QSF, QBA",1112475,2024-04-02T16:43:50+00:00
5,3521,KEWR,large_airport,Newark Liberty International Airport,40.692501,-74.168701,18.0,NaN,United States,US,...,1,KEWR,KEWR,EWR,EWR,http://www.panynj.gov/CommutingTravel/airports...,https://en.wikipedia.org/wiki/Newark_Liberty_I...,"Manhattan, New York City, NYC",1064475,2022-09-17T20:28:21+00:00
6,3488,KDFW,large_airport,Dallas Fort Worth International Airport,32.896801,-97.038002,607.0,NaN,United States,US,...,1,KDFW,KDFW,DFW,DFW,https://www.dfwairport.com/,https://en.wikipedia.org/wiki/Dallas/Fort_Wort...,QDF,1203175,2018-09-19T14:53:02+00:00
7,3631,KLAS,large_airport,Harry Reid International Airport,36.083361,-115.151817,2181.0,NaN,United States,US,...,1,KLAS,KLAS,LAS,LAS,https://www.harryreidairport.com/,https://en.wikipedia.org/wiki/Harry_Reid_Inter...,McCarran International Airport,1068475,2022-10-18T19:05:10+00:00
8,3670,KMCO,large_airport,Orlando International Airport,28.429399,-81.308998,96.0,NaN,United States,US,...,1,KMCO,KMCO,MCO,MCO,http://www.orlandoairports.net/,https://en.wikipedia.org/wiki/Orlando_Internat...,"Disney World,Epcot Center",1044075,2008-06-13T14:30:04+00:00
9,3486,KDEN,large_airport,Denver International Airport,39.861698,-104.672997,5431.0,NaN,United States,US,...,1,KDEN,KDEN,DEN,DEN,http://www.flydenver.com/,https://en.wikipedia.org/wiki/Denver_Internati...,"DVX, KVDX",1103275,2015-11-13T09:28:42+00:00


In [50]:
fl_sw_geo = fl_sw.merge(us_airports[['iata_code', 'latitude_deg', 'longitude_deg']], left_on='ORIGIN', right_on='iata_code', how='left')
fl_sw_geo.drop(['iata_code'], inplace=True, axis=1)
fl_sw_geo.rename(columns={'latitude_deg': 'ORIGIN_LATITUDE', 'longitude_deg': 'ORIGIN_LONGITUDE'}, inplace=True)
fl_sw_geo.head(10)

,FL_DATE,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,...,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT,ORIGIN_LATITUDE,ORIGIN_LONGITUDE
0,2019-01-01,WN,19393.0,1308.0,ATL,BNA,1615.0,1648.0,33.0,14.0,...,57.0,38.0,214.0,5.0,0.0,0.0,0.0,15.0,33.636700,-84.428101
1,2019-01-01,WN,19393.0,1719.0,PBI,BWI,700.0,655.0,-5.0,7.0,...,120.0,110.0,883.0,0.0,0.0,0.0,0.0,0.0,26.683201,-80.095596
2,2019-01-01,WN,19393.0,165.0,TPA,MDW,2010.0,2000.0,-10.0,8.0,...,146.0,132.0,997.0,0.0,0.0,0.0,0.0,0.0,27.975500,-82.533203
3,2019-01-01,WN,19393.0,738.0,MSY,LAS,2050.0,2103.0,13.0,8.0,...,207.0,197.0,1500.0,0.0,0.0,0.0,0.0,0.0,29.993401,-90.258003
4,2019-01-01,WN,19393.0,4.0,HOU,DAL,700.0,700.0,0.0,6.0,...,51.0,38.0,239.0,0.0,0.0,0.0,0.0,0.0,29.645399,-95.278900
5,2019-01-01,WN,19393.0,1403.0,DAL,DCA,1430.0,1441.0,11.0,27.0,...,160.0,127.0,1184.0,0.0,0.0,0.0,0.0,0.0,32.847099,-96.851799
6,2019-01-01,WN,19393.0,2201.0,ATL,AUS,940.0,947.0,7.0,20.0,...,153.0,129.0,813.0,0.0,0.0,0.0,0.0,0.0,33.636700,-84.428101
7,2019-01-01,WN,19393.0,2136.0,LAS,DEN,1220.0,1220.0,0.0,15.0,...,104.0,85.0,628.0,0.0,0.0,0.0,0.0,0.0,36.083361,-115.151817
8,2019-01-01,WN,19393.0,2253.0,MDW,TPA,2010.0,2015.0,5.0,26.0,...,160.0,131.0,997.0,0.0,0.0,15.0,0.0,0.0,41.785999,-87.752403
9,2019-01-01,WN,19393.0,946.0,STL,MSP,1040.0,1054.0,14.0,7.0,...,76.0,67.0,448.0,0.0,0.0,0.0,0.0,0.0,38.748697,-90.370003


In [62]:
fl_sw_geo_agg = fl_sw_geo.groupby("ORIGIN").agg(AVG_DELAY=("DEP_DELAY", "mean"), ORIGIN_LATITUDE=("ORIGIN_LATITUDE", "first"), ORIGIN_LONGITUDE=("ORIGIN_LONGITUDE", "first")).reset_index()
fl_sw_geo_agg.head(10)

,ORIGIN,AVG_DELAY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE
0,ABQ,9.219359,35.039976,-106.608925
1,ALB,7.426895,42.748299,-73.801697
2,AMA,4.878743,35.219398,-101.706001
3,ATL,10.905094,33.636700,-84.428101
4,AUS,10.552077,30.197535,-97.662015
5,BDL,7.661934,41.938510,-72.688066
6,BHM,8.791290,33.562901,-86.753502
7,BLI,13.863636,48.792801,-122.538002
8,BNA,9.711743,36.124500,-86.678200
9,BOI,5.072678,43.564400,-116.223000


In [64]:
fl_sw_delay_max, fl_sw_delay_min = fl_sw_geo_agg['AVG_DELAY'].max(), fl_sw_geo_agg['AVG_DELAY'].min()
fl_sw_geo_agg['MAX_AVG_DELAY'] = fl_sw_delay_max
fl_sw_geo_agg['MIN_AVG_DELAY'] = fl_sw_delay_min

fl_sw_delay_max, fl_sw_delay_min
fl_sw_geo_agg.head(10)

,ORIGIN,AVG_DELAY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,MAX_AVG_DELAY,MIN_AVG_DELAY
0,ABQ,9.219359,35.039976,-106.608925,19.60218,2.064133
1,ALB,7.426895,42.748299,-73.801697,19.60218,2.064133
2,AMA,4.878743,35.219398,-101.706001,19.60218,2.064133
3,ATL,10.905094,33.636700,-84.428101,19.60218,2.064133
4,AUS,10.552077,30.197535,-97.662015,19.60218,2.064133
5,BDL,7.661934,41.938510,-72.688066,19.60218,2.064133
6,BHM,8.791290,33.562901,-86.753502,19.60218,2.064133
7,BLI,13.863636,48.792801,-122.538002,19.60218,2.064133
8,BNA,9.711743,36.124500,-86.678200,19.60218,2.064133
9,BOI,5.072678,43.564400,-116.223000,19.60218,2.064133


In [65]:
gdf = gpd.GeoDataFrame(
    fl_sw_geo_agg[["ORIGIN", "AVG_DELAY", "MAX_AVG_DELAY", "MIN_AVG_DELAY"]],
    geometry=gpd.points_from_xy(fl_sw_geo_agg['ORIGIN_LONGITUDE'], fl_sw_geo_agg['ORIGIN_LATITUDE']),
    crs="EPSG:4326"
)

gdf.to_file('./data/fl_sw_agg.geojson', driver='GeoJSON')